In [ ]:
# test fc 
import torch 
xh = 3
xw = 4
th = 2
inner = 5
torch.set_printoptions(precision=10, linewidth=2000)
torch.random.manual_seed(20)
W0 = torch.nn.Parameter(torch.randn(inner, xh) )
b0 = torch.nn.Parameter(torch.randn(inner, 1) )
W1 = torch.nn.Parameter(torch.randn(th, inner) )
b1 = torch.nn.Parameter(torch.randn(th, 1) )

x = torch.ones(xh, xw)
y1 = torch.mm(W0, x) + b0
z1 = torch.sigmoid(y1)
y2 = torch.mm(W1, z1) + b1
z2 = torch.sigmoid(y2)

print("W0.shape", W0.shape, "\n values: ", W0)
print("b0.shape", b0.shape, "\n values: ", b0)
print("W1.shape", W1.shape, "\n values: ", W1)
print("b1.shape", b1.shape, "\n values: ", b1)

exp = torch.exp(z2)
s = exp / exp.sum(dim=0, keepdim=True)

# cross entropy:
t = torch.ones_like(s) * 5
#print("t: ", t, "\ns: ", s)
ce = -(t * torch.log(s)).sum()
#ce = (s - t).pow(2).mean()
ce.backward()
print("error: " , ce)

print("W0.grad.shape", W0.grad.shape, "\nW0.grad:" , W0.grad)
print("b0.grad.shape", b0.grad.shape, "\nb0.grad:" , b0.grad)
print("W1.grad.shape", W1.grad.shape, "\nW1.grad:" , W1.grad)
print("b1.grad.shape", b1.grad.shape, "\nb1.grad:" , b1.grad)

In [ ]:
#test attention
import torch
from tests.datagen import save_tensor_to_csv

embed_size = 7
seq_len = 5
torch.set_printoptions(precision=8, linewidth=2000)
torch.random.manual_seed(10)

Q = torch.nn.Parameter(torch.randn(seq_len, embed_size))
K = torch.nn.Parameter(torch.randn(seq_len, embed_size))
V = torch.nn.Parameter(torch.randn(seq_len, embed_size))

#print("Q: ", Q)
#print("K: ", K)
#print("V: ", V)

save_tensor_to_csv(Q, "data/attention_q.csv")
save_tensor_to_csv(K, "data/attention_k.csv")
save_tensor_to_csv(V, "data/attention_v.csv")

x = torch.ones(embed_size, seq_len)

q_ = x * .1 #torch.mm(Lq, x.t())
k_ = x * .2 #torch.mm(Lk, x.t())
v_ = x * .3 #torch.mm(Lv, x.t())

q = Q @ q_
k = K @ k_
v = V @ v_
v.retain_grad()
k.retain_grad()
q.retain_grad()
#print("q: ", q)
#print("k: ", k)
#print("v: ", v)

qkt = q @ k.t() / (embed_size ** (1 / 2))
qkt.retain_grad()
#print("qkt: ", qkt)

s = torch.softmax(qkt, dim=-1)
#s = qkt.exp() / qkt.exp().sum(dim=1, keepdim=True)
s.retain_grad()
#print("s: ", s)

att = s @ v
att.retain_grad()
#print("att", att)

err = (torch.ones_like(att) - att).pow(2).mean()
err.retain_grad()
err.backward()
print(err)

mse_grad = 2 * (att - torch.ones(seq_len, seq_len))/( seq_len * seq_len)

assert(torch.allclose(att.grad, mse_grad))
v_grad_in = torch.mm(s.t(), mse_grad)
assert(torch.allclose(v.grad, v_grad_in))
#print("v.grad:\n", v.grad)

s_grad_in = torch.mm(mse_grad, v.t())
assert(torch.allclose(s.grad, s_grad_in))
#print("s.grad:\n", s.grad)

V_grad = torch.mm(v_grad_in, v_.t())
assert(torch.allclose(V.grad, V_grad))
#print("V.grad:\n", V.grad)

#print("qkt.grad == s_grad_out in C++\n", qkt.grad)

torch.set_printoptions(precision=8)
k_grad_in = torch.mm(q.t(), qkt.grad).t() / (embed_size ** (1 / 2))
allclose = torch.allclose(k_grad_in, k.grad)
#print("k_grad_in:\n",k_grad_in)
#print("K.grad:\n", K.grad)


torch.set_printoptions(precision=10)
q_grad_in = torch.mm(qkt.grad, k) / (embed_size ** (1 / 2))
#print("q_grad_in:\n", q_grad_in)


print("att.grad:\n", att.grad)
print("s.grad:\n", s.grad)
print("V.grad:\n", v.grad)
print("qkt.grad:\n", qkt.grad)
